## Tools

In [ ]:
%pip install tensorflow[and-cuda]

In [ ]:
import numpy as np # math library
import pandas as pd # dataframe library
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # graphing library that builds ontop of mathplot
import tensorflow as tf # machine learning library
from tensorflow import keras # computer vision library from tensorflow 
import math
sns.set_style("darkgrid")

## Dataset
*Note: This is using a 5 class subset. The subset bird species are RUFOUS TREPE, HOUSE FINCH, D-ARNAUDS BARBET, OVENBIRD, ASIAN GREEN BEE EATER.
These 5 were choosen because they had the most available data out of the 500+ species in the orginial dataset.*

In [ ]:
# training subset = RUFOUS TREPE, HOUSE FINCH, D-ARNAUDS BARBET, OVENBIRD, ASIAN GREEN BEE EATER
#csv format: class id,filepaths(relative),labels,data set(e.g. training,valid or test),scientific name
data = pd.read_csv("birds.csv")
data.head(10)

## Basic Exploratory Data Analysis (EDA)
Worry about the EDA stuff after you get Naive Bayes working, it seems to matter little to it.

In [ ]:
# Grouping by data set and plotting histograms for each feature/label
for dataset, subset in data.groupby('data set'):
    plt.figure(figsize=(12, 10))
    subset['labels'].hist(alpha=0.5, label=dataset)
    plt.title(f'Histogram for data set: {dataset}')
    plt.xlabel('Labels')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

## Naive Bayes Implementation
Features = Pixel
P(Y=y), Y is the the target and is the precent chance of the target appearing. E.g. P(y) = 20%, if there are 20 out 100 sparrows in the data set of birds.
P(Feature_n | Y) = ???

In [ ]:
#Step 1 Computer vision!
# loop through dataframe and load image into computer vision
# constant for # bins(color groups) can +/- to +/- acc 
# Factor determines factor of reduction on color channels, e.g. 255 possible values per channel / 8 = 32 possible values per channel
# Factor is essetnially grouping similar colors together and controls how aggresive the grouping is.
FACTOR = 8
for index, row in data.iterrows():
    # read data set column of the given row
    dataset = row['data set']
    # skip over non-training data(data hygiene protection) to avoid trainning on valid/test data
    if dataset != 'train':
        continue
    # read filepath col of the given row
    filepath = row['filepaths']
    # read label
    label = row['labels']
    # load image into computer vision using native image size(this data set has an universal image size)
    image = tf.keras.utils.load_img(filepath)
    # the image is 224x224x3 (width x height x color_channels)
    image_array = tf.keras.utils.img_to_array(image,data_format='channels_last')
    
    # group similar colors together to reduce computation time based on a constant value that can be adjusted
    image_array = image_array // FACTOR # Reduce from 255x255x255 to 32x32x32, much easier for bayes
    # create enough color bins for each grouped color dynimcally.
    bins_num = math.ceil(255 / FACTOR)
    
    # 3D Slice colstart:colstop, rowstart:rowstop, depthstart:depthstop
    # Splitting the image_array into R, G, B channels
    red_channel = image_array[:, :, 0]
    green_channel = image_array[:, :, 1]
    blue_channel = image_array[:, :, 2]
    
    # Create histograms data strucutre for each channel group (255 // 8 = 32)
    # x_hist = array, x_bins = bins start indexes in said array
    # note, since each color group has it's own bin, each index i corresponds to bin i making x_bins variable redundant.
    red_hist, red_bins = np.histogram(red_channel, bins=bins_num, range=(0, bins_num))
    green_hist, green_bins = np.histogram(green_channel, bins=bins_num, range=(0, bins_num))
    blue_hist, blue_bins = np.histogram(blue_channel, bins=bins_num, range=(0, bins_num))
    
    # Plotting histograms for each channel
    plt.figure(figsize=(8, 6))

    plt.hist(red_channel.flatten(), bins=bins_num, range=(0, bins_num), color='red', alpha=0.5, label='Red')
    plt.hist(green_channel.flatten(), bins=bins_num, range=(0, bins_num), color='green', alpha=0.5, label='Green')
    plt.hist(blue_channel.flatten(), bins=bins_num, range=(0, bins_num), color='blue', alpha=0.5, label='Blue')

    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.title('Histogram of Pixel Color Values (R, G, B)')
    plt.legend()
    plt.show()
    #TODO: Remove this break!
    break

# Naive Bayes Function
Function should be implemented in the block above. See planning doc for how to assign prob values.

In [ ]:
# take in the pixel color channel histograms derived from a given image
# take in target, feature name,
def Naive_Bayes(redHisto,greenHisto,blueHisto):
    # all data set images are in 224 * 224
    TOTAL_PIXELS = 224 * 224
    #P(Feature) = pixels in bin / total pixels(224x224)
    #P(Y) = # Y bird in data set / # total birds in dataset(need to check / dynamically cal earlier)
    #P(Feature | Y) = (Occurrences of color_bin(Feature) in images of Y) / (Total pixel occurrences in histograms of Y) 
    # The above requires some running sums.
    # Those are the 3 probs needed for naive bayes
    # Need to consider partial 
    pass

Placeholder for now. Seems like sklearn.metrics is what we are meant to be use. Which means sklearn intergration somewhat.

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=.2, random_state=41)

X_test = test.iloc[:,:-1].values # test features
Y_test = test.iloc[:,-1].values # test labels
Y_pred = naive_bayes_categorical(train, X=X_test, Y="diagnosis")

from sklearn.metrics import confusion_matrix, f1_score
print(confusion_matrix(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred))